In [36]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from configobj import ConfigObj
from itertools import chain
import os, glob
import shutil
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
import astropy.constants as const
import dustmaps
from dustmaps.sfd import SFDQuery
from dust_extinction.parameter_averages import CCM89, G23

In [37]:
#define paramters to be modified in default config file
#for any section that requires paramters to be edited, include it in 'sed_modules_params'
CIGALE_PARAMS = {'data_file': 'ptf_ccsne_cigale.txt',
                 'sed_modules': ['sfhdelayed', 'bc03', 'nebular', 'dustatt_calzleit', 'dale2014', 'redshifting'],
                 'analysis_method': "pdf_analysis",
                 'sed_modules_params': {
                     'sfhdelayed': {
                        'tau_main': ['250', '500', '1000', '2000', '4000', '6000', '8000'],
                        'age_main': ['250', '500', '1000', '2000', '4000', '8000', '12000'],
                     },
                     'bc03': {
                         'imf': 1,
                         'metallicity': ['0.0001', '0.0004', '0.004', '0.008', '0.02', '0.05'],
                     },
                     'dustatt_calzleit': {
                         'E_BVs_young': ['0.0', '0.2', '0.3', '0.5', '0.8', '1.0', '1.5', '2.0', '3.0'],
                         'E_BVs_old_factor': ['0.3', '0.5', '1.0'],
                         'uv_bump_amplitude': ['0.0', '1.0', '2.0', '3.0'],
                         'powerlaw_slope': ['-0.13', '-0.2', '-0.5']
                     },
                     'dale2014': {
                         'alpha': ['1.0', '1.5', '2.0', '2.5'],
                     }
                 },
                 'analysis_params': {
                     'save_best_sed': True,
                 }}

In [38]:
CIGALE_PTF_FILTERS = ['GALEX_FUV', 'GALEX_NUV', 'SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z',
                      'PANSTARRS_g', 'PANSTARRS_r', 'PANSTARRS_i', 'PANSTARRS_z', 'PANSTARRS_y',
                      'CFHT_u', 'CFHT_g', 'CFHT_r', 'CFHT_i', 'CFHT_z', 'LS_g', 'LS_r', 'LS_z',
                      '2MASS_J', '2MASS_H', '2MASS_Ks', 'WISE_W1', 'WISE_W2']

In [39]:
pivot_lambda = [153.51*u.nm, 230.08*u.nm, 350.10*u.nm, 476.31*u.nm, 624.70*u.nm, 771.83*u.nm, 1.08*u.um,
                484.91*u.nm, 620.12*u.nm, 753.50*u.nm, 867.42*u.nm, 962.78*u.nm,
                364.75*u.nm, 474.15*u.nm, 637.55*u.nm, 767.88*u.nm, 900.66*u.nm, 480.85*u.nm, 641.76*u.nm, 916.89*u.nm,
                1.24*u.um, 1.65*u.um, 2.16*u.um, 3.37*u.um, 4.62*u.um]
CIGALE_PTF_LAMBDA = dict(zip(CIGALE_PTF_FILTERS, pivot_lambda))

In [40]:
def prep_cigale(params):

    subprocess.run(["pcigale", "init"])
    configfile = 'pcigale.ini'

    #read in configfile
    config = ConfigObj(configfile, encoding = 'utf-8')
    config.filename = configfile

    #edit params
    config['data_file'] = params['data_file']
    config['sed_modules'] = params['sed_modules']
    config['analysis_method'] = params['analysis_method']
    config.write()
    
    #generate config file to run cigale
    try:
        subprocess.run(["pcigale", "genconf"])
    except:
        raise ValueError("Could not create config file - check defaults")
    
    #read configfile again as it has been modified by genconf
    config = ConfigObj(configfile, encoding = 'utf-8')
    config.filename = configfile
    
    #genconf will overwrite any edits, so modify paramters at the end
    for module_ in params['sed_modules']:
        if module_ in params['sed_modules_params'].keys():
            mod = params['sed_modules_params'][module_]
            for key, value in mod.items():
                config['sed_modules_params'][module_][key] = value

    if 'analysis_params' in params.keys():
        for key, value in params['analysis_params'].items():
            config['analysis_params'][key] = value
    config.write()

    #verify if config file is ok
    try:
        subprocess.run(["pcigale", "check"])
    except:
        raise ValueError("Config file in incorrect")
    
    return config

In [41]:
def run_cigale(configfile, params, outdir, plot=False):
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    #copy everything to outdir
    shutil.copy(configfile, os.path.join(outdir, configfile))
    shutil.copy(configfile + '.spec', os.path.join(outdir, configfile + '.spec'))
    shutil.copy(params['data_file'], os.path.join(outdir, params['data_file']))

    cwd = os.getcwd()
    os.chdir(outdir)

    #run cigale
    subprocess.run(["pcigale", "run"])
    if plot:
        subprocess.run(["pcigale-plots", "sed"])
    os.chdir(cwd)

In [42]:
def read_ptf_sample(filename):
    try:
        ptf_sne = pd.read_csv(filename, header = None, names=['SN', 'telescope', 'instrument', 'filter', 'mag', 'magerr', 'ref'])
        ptf_sne['SN'] = ptf_sne['SN'].str.strip()
        ptf_sne['telescope'] = ptf_sne['telescope'].str.strip()
        ptf_sne['instrument'] = ptf_sne['instrument'].str.strip()
        ptf_sne['filter'] = ptf_sne['filter'].str.strip()
        ptf_sne['ref'] = ptf_sne['ref'].str.strip()
        
        return ptf_sne
    except:
        raise ValueError('Reformat input file with correct whitespaces and delimiters')

In [87]:
def read_ptf_redshifts(filename):
    try:
        redshifts = pd.read_csv(filename, header = None, usecols = [0, 1, 2, 5, 6, 7, 8, 9, 10],  names = ['SN', 'Type', 'z', 'RAh', 'RAm', 'RAs', 'Decd', 'Decm', 'Decs'])
        redshifts['RA'] = redshifts['RAh'].astype(str) + ':' + redshifts['RAm'].astype(str) + ':' + redshifts['RAs'].astype(str)
        redshifts['Dec'] = redshifts['Decd'].astype(str) + ':' + redshifts['Decm'].astype(str) + ':' + redshifts['Decs'].astype(str)
        redshifts = redshifts.drop(columns=['RAh', 'RAm', 'RAs', 'Decd', 'Decm', 'Decs'])
        redshifts['SN'] = redshifts['SN'].str.strip()
        redshifts['Type'] = redshifts['Type'].str.strip()

        return redshifts
    
    except:
        raise ValueError('Reformat input file with correct whitespaces and delimiters')

In [107]:
def mw_extinction(sn, redshifts, filters):
    # Get SN coordinates from redshifts table
    ra = redshifts[redshifts['SN'] == sn]['RA']
    dec = redshifts[redshifts['SN'] == sn]['Dec']
    coord = SkyCoord(ra=ra, dec=dec, unit = (u.hourangle, u.deg), frame='icrs')

    # Query SFD dust map for E(B-V)
    sfd = SFDQuery()
    ebv = sfd(coord)[0]
    R_V = 3.1

    # Use extinction law from Gordon+23
    ext = G23(Rv = R_V)
    out_ext = dict.fromkeys(filters)
    for flt_ in filters:
        wv = CIGALE_PTF_LAMBDA[flt_]
        a_lambda = ext(wv)
        A_lambda = a_lambda * ebv
        print(f'A_lambda for {sn} in {flt_} is {A_lambda}')
        out_ext[flt_] = A_lambda

    return out_ext

In [108]:
def get_sn_data(sn, ptf_sne, redshifts):

    # Read mags, redshift and filters
    dat = ptf_sne[ptf_sne['SN'].str.lower() == sn.lower()]
    z = redshifts[redshifts['SN'].str.lower() == sn.lower()]['z'].iloc[0]
    filters = dat['telescope'] + '_' + dat['filter']
    mag, err = np.array(dat['mag']), np.array(dat['magerr'])

    maglist, errlist = [], []
    for f in CIGALE_PTF_FILTERS:
        if f in list(filters):
            idx = list(filters).index(f)
            # Correct photometry for MW extinction
            mwext = mw_extinction(sn, redshifts, [f])[f]
            print(f'MW extinction for {sn} in {f}: {mag[idx]} -> {mag[idx] - mwext}')
            maglist.append(mag[idx] - mwext); errlist.append(err[idx])
        else:
            maglist.append(np.nan); errlist.append(np.nan)

    maglist, errlist = np.array(maglist), np.array(errlist)

    # Convert magnitudes to flux in mJy as required by CIGALE
    flux = 10**((maglist - 8.90)/-2.5)*1000
    fluxerr = errlist * flux * np.log(10)/2.5

    # Output string for CIGALE data file
    line = [f'{i} {j}' for i, j in zip(flux, fluxerr)]
    line = ' '.join(line)
    row = f'{sn} {z} ' + line

    return row

In [114]:
config = prep_cigale(CIGALE_PARAMS)

╭──────────────────────────────────────────────────────────────────────────────╮
│                      Code Investigating GALaxy Emission                      │
│                Boquien et al. (2019) (]8;id=950545;https://cigale.lam.fr\https://cigale.lam.fr]8;;\)                 │
│         CIGALE version: 2025.0 — Python version: 3.12.0 — Platform:          │
│                              macosx-11.1-arm64                               │
╰───────────────────────────���──────────────────────────────────────────────────╯


                                  SED modules                                   
╭──────────────────────┬───────────────────────────────────────────────────────╮
│ SFH                  │ None. Options are: sfh2exp, sfh_buat08, sfhdelayed,   │
│                      │ sfhdelayedbq, sfhfromfile, sfhperiodic.               │
│ SSP                  │ None. Options are: bc03, bpassv2, cb19, m2005.        │
│ nebular              │ None. Options are: nebular.         

In [115]:
run_cigale('pcigale.ini', CIGALE_PARAMS, 'ptfccsne', plot=True)

╭──────────────────────────────────────────────────────────────────────────────╮
│                      Code Investigating GALaxy Emission                      │
│                Boquien et al. (2019) (]8;id=331039;https://cigale.lam.fr\https://cigale.lam.fr]8;;\)                 │
│         CIGALE version: 2025.0 — Python version: 3.12.0 — Platform:          │
│                              macosx-11.1-arm64                               │
╰───────────────────────────���──────────────────────────────────────────────────╯


                              General information                               
╭───────────────────┬──────────────────────────────────────────────────────────╮
│ Data file         │ ptf_ccsne_cigale.txt                                     │
│ Parameters file   │ None                                                     │
│ Number of objects │ 8                                                        │
│ Redshift          │ 0.02 to 0.17                           

In [109]:
ptf_sne = read_ptf_sample('ptf_ccsne.csv')
ptf_sne = ptf_sne[ptf_sne['ref'] == 'This paper']

redshifts = read_ptf_redshifts('ptf_ccsne_redshifts.csv')

In [111]:
rows = []
sne = ['PTF09awk', 'PTF10ad', 'PTF10acsq', 'PTF11kaa', 'iPTF13aqj', 'iPTF15dtg', 'iPTF16hfb', 'iPTF17zg']
for sn in sne:
    rows.append(get_sn_data(sn, ptf_sne, redshifts))

A_lambda for PTF09awk in GALEX_FUV is 0.05812651291489601
MW extinction for PTF09awk in GALEX_FUV: 20.24 -> 20.181873487085102
A_lambda for PTF09awk in GALEX_NUV is 0.06091223657131195
MW extinction for PTF09awk in GALEX_NUV: 20.03 -> 19.96908776342869
A_lambda for PTF09awk in SDSS_u is 0.03359860181808472
MW extinction for PTF09awk in SDSS_u: 19.2 -> 19.166401398181915
A_lambda for PTF09awk in SDSS_g is 0.025252578780055046
MW extinction for PTF09awk in SDSS_g: 18.16 -> 18.134747421219945
A_lambda for PTF09awk in SDSS_r is 0.017997190356254578
MW extinction for PTF09awk in SDSS_r: 17.8 -> 17.782002809643746
A_lambda for PTF09awk in SDSS_i is 0.013069268316030502
MW extinction for PTF09awk in SDSS_i: 17.51 -> 17.49693073168397
A_lambda for PTF09awk in SDSS_z is 0.00718860374763608
MW extinction for PTF09awk in SDSS_z: 17.43 -> 17.422811396252364
A_lambda for PTF09awk in PANSTARRS_g is 0.024832414463162422
MW extinction for PTF09awk in PANSTARRS_g: 18.15 -> 18.125167585536836
A_lambda f

In [113]:
with open('ptf_ccsne_cigale.txt', 'a') as f:
    f.write('\n'.join(rows))

In [ ]:
awkmag = np.array([20.00, 19.90, 19.55, 18.53, 18.24, 17.96, 17.93, 18.75, 19.52])
awkerr = np.array([0.17, 0.06, 0.11, 0.03, 0.06, 0.09, 0.09, 0.14, 0.12])
awkflux = 10**((awkmag - 8.90)/-2.5)*1000
awkfluxerr = awkerr * awkflux * np.log(10)/2.5

In [241]:
awkerr

array([ 1.4e-01,  1.7e-01,  8.0e-02,  1.1e-01,  1.0e-01,  1.5e-01,
        3.6e-01, -9.9e+01, -9.9e+01])

In [116]:
hdul = fits.open('ptfccsne/out/results.fits')
results = Table(hdul[1].data)

In [117]:
results

id,bayes.agn.fracAGN_dale2014,bayes.agn.fracAGN_dale2014_err,bayes.attenuation.E_BVs.nebular.emission_old,bayes.attenuation.E_BVs.nebular.emission_old_err,bayes.attenuation.E_BVs.nebular.emission_young,bayes.attenuation.E_BVs.nebular.emission_young_err,bayes.attenuation.E_BVs.stellar.old,bayes.attenuation.E_BVs.stellar.old_err,bayes.attenuation.E_BVs.stellar.young,bayes.attenuation.E_BVs.stellar.young_err,bayes.attenuation.ebvs_old_factor,bayes.attenuation.ebvs_old_factor_err,bayes.attenuation.galex.FUV,bayes.attenuation.galex.FUV_err,bayes.attenuation.generic.bessell.B,bayes.attenuation.generic.bessell.B_err,bayes.attenuation.generic.bessell.V,bayes.attenuation.generic.bessell.V_err,bayes.attenuation.powerlaw_slope,bayes.attenuation.powerlaw_slope_err,bayes.attenuation.uv_bump_amplitude,bayes.attenuation.uv_bump_amplitude_err,bayes.attenuation.uv_bump_wavelength,bayes.attenuation.uv_bump_wavelength_err,bayes.attenuation.uv_bump_width,bayes.attenuation.uv_bump_width_err,bayes.dust.alpha,bayes.dust.alpha_err,bayes.nebular.f_dust,bayes.nebular.f_dust_err,bayes.nebular.f_esc,bayes.nebular.f_esc_err,bayes.nebular.lines_width,bayes.nebular.lines_width_err,bayes.nebular.logU,bayes.nebular.logU_err,bayes.nebular.ne,bayes.nebular.ne_err,bayes.nebular.zgas,bayes.nebular.zgas_err,bayes.sfh.age,bayes.sfh.age_err,bayes.sfh.age_burst,bayes.sfh.age_burst_err,bayes.sfh.age_main,bayes.sfh.age_main_err,bayes.sfh.f_burst,bayes.sfh.f_burst_err,bayes.sfh.tau_burst,bayes.sfh.tau_burst_err,bayes.sfh.tau_main,bayes.sfh.tau_main_err,bayes.stellar.age_m_star,bayes.stellar.age_m_star_err,bayes.stellar.imf,bayes.stellar.imf_err,bayes.stellar.metallicity,bayes.stellar.metallicity_err,bayes.stellar.old_young_separation_age,bayes.stellar.old_young_separation_age_err,bayes.universe.age,bayes.universe.age_err,bayes.universe.luminosity_distance,bayes.universe.luminosity_distance_err,bayes.universe.redshift,bayes.universe.redshift_err,bayes.attenuation.nebular.emission_old,bayes.attenuation.nebular.emission_old_err,bayes.attenuation.nebular.emission_young,bayes.attenuation.nebular.emission_young_err,bayes.attenuation.stellar.old,bayes.attenuation.stellar.old_err,bayes.attenuation.stellar.young,bayes.attenuation.stellar.young_err,bayes.dust.luminosity,bayes.dust.luminosity_err,bayes.sfh.integrated,bayes.sfh.integrated_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.sfh.sfr100Myrs,bayes.sfh.sfr100Myrs_err,bayes.sfh.sfr10Myrs,bayes.sfh.sfr10Myrs_err,bayes.stellar.lum,bayes.stellar.lum_err,bayes.stellar.lum_ly,bayes.stellar.lum_ly_err,bayes.stellar.lum_ly_old,bayes.stellar.lum_ly_old_err,bayes.stellar.lum_ly_young,bayes.stellar.lum_ly_young_err,bayes.stellar.lum_old,bayes.stellar.lum_old_err,bayes.stellar.lum_young,bayes.stellar.lum_young_err,bayes.stellar.m_gas,bayes.stellar.m_gas_err,bayes.stellar.m_gas_old,bayes.stellar.m_gas_old_err,bayes.stellar.m_gas_young,bayes.stellar.m_gas_young_err,bayes.stellar.m_star,bayes.stellar.m_star_err,bayes.stellar.m_star_old,bayes.stellar.m_star_old_err,bayes.stellar.m_star_young,bayes.stellar.m_star_young_err,bayes.stellar.n_ly,bayes.stellar.n_ly_err,bayes.stellar.n_ly_old,bayes.stellar.n_ly_old_err,bayes.stellar.n_ly_young,bayes.stellar.n_ly_young_err,bayes.2mass.H,bayes.2mass.H_err,bayes.2mass.J,bayes.2mass.J_err,bayes.2mass.Ks,bayes.2mass.Ks_err,bayes.cfht.megaprime.g,bayes.cfht.megaprime.g_err,bayes.cfht.megaprime.i,bayes.cfht.megaprime.i_err,bayes.cfht.megaprime.r,bayes.cfht.megaprime.r_err,bayes.cfht.megaprime.u,bayes.cfht.megaprime.u_err,bayes.cfht.megaprime.z,bayes.cfht.megaprime.z_err,bayes.ctio.ls.g,bayes.ctio.ls.g_err,bayes.ctio.ls.r,bayes.ctio.ls.r_err,bayes.ctio.ls.z,bayes.ctio.ls.z_err,bayes.galex.FUV,bayes.galex.FUV_err,bayes.galex.NUV,bayes.galex.NUV_err,bayes.panstarrs.ps1.g,bayes.panstarrs.ps1.g_err,bayes.panstarrs.ps1.i,bayes.panstarrs.ps1.i_err,bayes.panstarrs.ps1.r,bayes.panstarrs.ps1.r_err,bayes.panstarrs.ps1.y,bayes.panstarrs.ps1.y_err,bayes.panstarrs.ps1.z,bayes.panstarrs.ps1.z_err,bayes.sloan.sdss.gprime,bay

In [120]:
m_star, m_star_err = results['bayes.stellar.m_star'].value[-1], results['bayes.stellar.m_star_err'].value[-1]
log_m_star, log_m_star_err = np.log10(m_star), m_star_err/m_star
print(log_m_star, log_m_star_err)

10.118445588896885 0.36631751623798464


In [121]:
sfh, sfh_err = results['bayes.sfh.sfr'].value[-1], results['bayes.sfh.sfr_err'].value[-1]
log_sfh, log_sfh_err = np.log10(sfh), sfh_err/sfh
print(log_sfh, log_sfh_err)

0.30319156086392923 0.8123062442944401


In [122]:
age, age_err = results['bayes.sfh.age'].value[-1]*1e6, results['bayes.sfh.age_err'].value[-1]*1e6
log_age, log_age_err = np.log10(age), age_err/age
print(log_age, log_age_err)

9.839026256957712 0.5177775405099265
